In [1]:
import timeit
start=timeit.default_timer()

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Load files

In [7]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [8]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    text=nlp(text)
    return text

In [9]:
sentences=pd.DataFrame()
for i in range(0,len(filename)):
    text=load_text(filename[i])
    author=filename[i].replace('.txt','')
    text_sents=[[sent,author] for sent in text.sents]
    text_df=pd.DataFrame(text_sents)
    sentences=sentences.append(text_df)

In [10]:
sentences

,0,1
0,"(ï»¿The, Project, Gutenberg, eBook, ,, And, Th...",brady
1,"(Illustrated, by, Walter, B., Everett, \n\n\n)",brady
2,"(This, eBook, is, for, the, use, of, anyone, a...",brady
3,"(You, may, copy, it, ,, give, it, away, or, \n...",brady
4,"(Title, :)",brady
5,"(And, Thus, He, Came, \n\n)",brady
6,"(Author, :, Cyrus, Townsend, Brady, \n\n, Rele...",brady
7,"(:, January, 5, ,, 2005, )",brady
8,"([, eBook, #, 14606, ], \n\n)",brady
9,"(Language, :)",brady


In [11]:
# Utility function to create a list of the 50 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
      
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(50)]

In [12]:
common_words=set()
for i in range(0,len(filename)):
    text=load_text(filename[i])
    words=bag_of_words(text)
    common_words.update(words)

In [13]:
common_words=[i.strip() for i in common_words]


In [14]:
common_words=[c for c in common_words if c.isalpha()]

In [15]:
STOP_WORDS=list(STOP_WORDS)

In [16]:
common_words=[x for x in common_words if x not in STOP_WORDS]

In [17]:
stop=['copyright','good','great','gutenberg','project',',oh','th',
     'christmas','tree','mary','claus']
common_words=[x for x in common_words if x not in stop]

In [18]:
print(common_words)

['santy', 'langshaw', 'dollar', 'philip', 'charles', 'work', 'big', 'khwis', 'kris', 'thou', 'send', 'joan', 'alice', 'life', 'come', 'electronic', 'year', 'thee', 'die', 'hand', 'away', 'father', 'man', 'thin', 'right', 'sing', 'baby', 'figure', 'rod', 'bishop', 'love', 'archive', 'cry', 'animal', 'hugh', 'gratz', 'shall', 'child', 'marietta', 'nan', 'face', 'arm', 'find', 'oh', 'money', 'foundation', 'wife', 'look', 'write', 'long', 'yes', 'cousin', 'tm', 'york', 'guy', 'want', 'mother', 'girl', 'nathan', 'heaven', 'include', 'read', 'use', 'toober', 'chlosis', 'like', 'house', 'eye', 'rhoda', 'letter', 'santa', 'fall', 'present', 'heart', 'cave', 'flannery', 'hear', 'colonel', 'thing', 'winfried', 'yard', 'chicken', 'think', 'priest', 'agreement', 'george', 'fernald', 'fire', 'aunt', 'fool', 'day', 'god', 'copy', 'door', 'term', 'boy', 'turn', 'donation', 'castle', 'pay', 'clytie', 'provide', 'oak', 'know', 'old', 'night', 'lord', 'whisper', 'forest', 'clotilde', 'distribute', 'ligh

In [19]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(index=range(len(sentences)),columns=common_words)
    df['text_sentence'] = sentences[0].values
    df['text_source'] = sentences[1].values
    df.loc[:,common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
       
    return df

In [20]:
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000


In [21]:
word_counts.shape

(7224, 130)

In [22]:
word_counts

,santy,langshaw,dollar,philip,charles,work,big,khwis,kris,thou,...,manger,woman,dog,set,little,bug,new,state,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ï»¿The, Project, Gutenberg, eBook, ,, And, Th...",brady
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Illustrated, by, Walter, B., Everett, \n\n\n)",brady
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(This, eBook, is, for, the, use, of, anyone, a...",brady
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(You, may, copy, it, ,, give, it, away, or, \n...",brady
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Title, :)",brady
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(And, Thus, He, Came, \n\n)",brady
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Author, :, Cyrus, Townsend, Brady, \n\n, Rele...",brady
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(:, January, 5, ,, 2005, )",brady
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"([, eBook, #, 14606, ], \n\n)",brady
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Language, :)",brady


In [23]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.25,
                                                    random_state=0)

### Random Forest

In [25]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(X_train, y_train)
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.4736064968623108

Test set score: 0.3327796234772979


### Logistic Regression

In [26]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.38870431893687707

Test set score: 0.34662236987818384


In [27]:
stop=timeit.default_timer()
time_used=stop-start
print("Time Used:",time_used)

Time Used: 185.42761420000002
